Reference: 
- https://github.com/NVIDIA/DeepRecommender/blob/master/reco_encoder/model/model.py
- https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py
- https://users.cecs.anu.edu.au/~akmenon/papers/autorec/autorec-paper.pdf

In [1]:
import pandas as pd

In [2]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader

### 1. Implement model.

In [3]:
def MSEloss(inputs, targets):
    mask = targets != 0
    num_ratings = torch.sum(mask.float())
    criterion = nn.MSELoss(reduction='sum')
    return criterion(inputs * mask.float(), targets)

In [4]:
class AutoEncoder(nn.Module):
    def __init__(self, size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(size, 64),
            nn.ReLU(True),
            nn.Linear(64, 16),
            nn.ReLU(True),
        )

        self.decoder = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(True), 
            nn.Linear(64, size), 
            nn.ReLU(True),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### 2. Load and process data.

In [5]:
raw = pd.read_csv("datasets/movie_ratings.csv")
raw.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
df = pd.pivot_table(raw, values="rating", index="movieId", columns="userId", fill_value=0.0)
df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4,0.0,0.0,0,4,0,4.5,0,0,0.0,...,4.0,0,4,3,4.0,2.5,4,2.5,3,5.0
2,0,0.0,0.0,0,0,4,0.0,4,0,0.0,...,0.0,4,0,5,3.5,0.0,0,2.0,0,0.0
3,4,0.0,0.0,0,0,5,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,2.0,0,0.0
4,0,0.0,0.0,0,0,3,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
5,0,0.0,0.0,0,0,5,0.0,0,0,0.0,...,0.0,0,0,3,0.0,0.0,0,0.0,0,0.0


In [7]:
tensor = torch.from_numpy(df.values)

### 3. Train model

In [8]:
num_epochs = 200
batch_size = 128
learning_rate = 1e-3
weight_decay=1e-5

In [9]:
data_small = DataLoader(tensor[:10, :], batch_size=2)

In [10]:
data = DataLoader(tensor, batch_size=batch_size)

In [11]:
model = AutoEncoder(size=tensor.shape[1])
criterion = MSEloss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [12]:
for epoch in range(num_epochs):
    for inputs in data:
        # forward
        targets = model(inputs.float())
        loss = criterion(inputs.float(), targets.float())
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # log
    print(f'epoch [{epoch + 1}/{num_epochs}], loss:{round(float(loss), 3)}')

epoch [1/200], loss:1892.91
epoch [2/200], loss:1927.505
epoch [3/200], loss:1884.451
epoch [4/200], loss:1709.866
epoch [5/200], loss:1560.298
epoch [6/200], loss:1421.942
epoch [7/200], loss:1292.482
epoch [8/200], loss:1199.969
epoch [9/200], loss:1130.861
epoch [10/200], loss:1052.132
epoch [11/200], loss:964.171
epoch [12/200], loss:878.151
epoch [13/200], loss:846.915
epoch [14/200], loss:775.89
epoch [15/200], loss:756.796
epoch [16/200], loss:737.935
epoch [17/200], loss:728.605
epoch [18/200], loss:683.943
epoch [19/200], loss:644.844
epoch [20/200], loss:643.204
epoch [21/200], loss:622.143
epoch [22/200], loss:613.358
epoch [23/200], loss:590.139
epoch [24/200], loss:575.745
epoch [25/200], loss:567.418
epoch [26/200], loss:581.667
epoch [27/200], loss:571.942
epoch [28/200], loss:561.083
epoch [29/200], loss:569.798
epoch [30/200], loss:532.377
epoch [31/200], loss:527.207
epoch [32/200], loss:512.491
epoch [33/200], loss:503.765
epoch [34/200], loss:491.548
epoch [35/200],

### 4. Analysis
The output is the rating prediction, and we need to remove the movied user already watched when generating a recommend list.

In [13]:
# Test the first row and its first 8 elements.
for x, y in zip(tensor[0, :8], model(tensor[0, :].float())[:8]):
    print(f"Input is {x} and output is {round(float(y), 1)}!")

Input is 4.0 and output is 1.8!
Input is 0.0 and output is 0.0!
Input is 0.0 and output is 0.0!
Input is 0.0 and output is 0.0!
Input is 4.0 and output is 2.8!
Input is 0.0 and output is 0.0!
Input is 4.5 and output is 2.8!
Input is 0.0 and output is 0.0!


In [14]:
# List top 10 recommend list of movie 1
result = []
for x, y, i in zip(tensor[0, :], model(tensor[0, :].float()), range(1, tensor.shape[1] + 1)):
    if x > 0.0001:
        continue
    result.append((i, round(float(y), 3)))
    
sorted(result, key=lambda x: x[1], reverse=True)[:10]

[(84, 3.731),
 (122, 3.102),
 (415, 2.874),
 (426, 2.603),
 (58, 2.488),
 (198, 2.325),
 (377, 2.285),
 (376, 2.26),
 (387, 2.047),
 (37, 1.996)]

In [15]:
# The recommend list length plus the viewed list length equals to total number of movies. 
assert len(result) + sum(float(x) != 0 for x in tensor[0, :]) == tensor.shape[1]